In [52]:
%run -i 'setup.py'

import os
import json

import pandas as pd
import numpy as np

from src.settings import BASE_DIR

oecd = os.path.join(BASE_DIR, 'data/country_stats/oecd/NAAG_13102019054548637.csv')

In [46]:
# General Country Data from OECD

country_df = pd.read_csv(oecd)

data_labels = country_df.loc[:, ['INDICATOR', 'Indicator']].drop_duplicates().set_index('INDICATOR')

country_df = (
    country_df
    .drop(
        ['TIME', 'INDICATOR', 'Unit Code', 'Unit', 'PowerCode Code', 'PowerCode',
         'Reference Period Code', 'Reference Period', 'Flag Codes', 'Flags'],
        axis=1
    )
    .rename(columns={'LOCATION': 'Code'})
    .pivot_table(index=['Code', 'Country', 'Time'], columns=['Indicator'], values=['Value'])
    .droplevel(level=0, axis=1)
    .groupby(level=0)
    .ffill()
    # Social benefits seems to double-up with other budgets, because removing it gets the budget sum
    # close to the total budget figures
    .drop(['Code', 'Social benefits and social transfers in kind, percentage of GDP'], axis=1)
    .dropna()
)

country_df

General government expenditure by function, defence, percentage of GDP  \
Code Country       Time                                                                           
AUS  Australia     2000                                           1.506242                        
                   2001                                           1.543646                        
                   2002                                           1.589343                        
                   2003                                           1.483208                        
                   2004                                           1.412989                        
                   2005                                           1.418564                        
                   2006                                           1.459700                        
                   2007                                           1.395296                        
                   2008                                           1.438644                        
                   2009                                           1.469938                        
                   2010                                           1.369949                        
                   2011                                           1.394237                        
                   2012                                           1.329357                        
                   2013                                           1.356371                        
                   2014                                           1.451312                        
                   2015                                           1.574530                        
                   2016                                           1.574530                        
                   2017                                           1.574530                        
                   2018                                           1.574530                        
AUT  Austria       2000                                           0.863822                        
                   2001                                           0.820097                        
                   2002                                           0.783628                        
                   2003                                           0.823458                        
                   2004                                           0.827210                        
                   2005                                           0.782127                        
                   2006                                           0.713919                        
                   2007                                           0.755421                        
                   2008                                           0.896471                        
                   2009                                           0.707427                        
                   2010                                           0.643255                        
...                                                                    ...                        
SWE  Sweden        2008                                           1.474932                        
                   2009                                           1.481254                        
                   2010                                           1.515229                        
                   2011                                           1.446391                        
                   2012                                           1.402082                        
                   2013                                           1.459960                        
                   2014                                           1.282948                        
                   2015                                          

In [50]:
gdp = country_df['Gross domestic product (GDP), current PPPs, billions US dollars']
budgets = country_df.drop('Gross domestic product (GDP), current PPPs, billions US dollars', axis=1)

budgets_dollar = (
    budgets
    .apply(lambda col: np.asarray(col) * np.asarray(gdp))
    .round(2)
    .loc[(slice(None), slice(None), 2018), :]
    .reset_index()
)
budgets_dollar

,Code,Country,Time,"General government expenditure by function, defence, percentage of GDP","General government expenditure by function, economic affairs, percentage of GDP","General government expenditure by function, education, percentage of GDP","General government expenditure by function, environment protection, percentage of GDP","General government expenditure by function, general public services, percentage of GDP","General government expenditure by function, health, percentage of GDP","General government expenditure by function, housing and community amenities, percentage of GDP","General government expenditure by function, public order and safety, percentage of GDP","General government expenditure by function, recreation, culture and religion, percentage of GDP","General government expenditure by function, social protection, percentage of GDP","Gross debt of general government, percentage of GDP","Total expenditure of general government, percentage of GDP","Total general government (GG) revenue, percentage of GDP"
0,AUS,Australia,2018,2130.69,4899.34,7151.97,1189.57,6112.27,9476.24,799.88,2383.37,979.71,13786.13,89783.66,49678.94,48541.24
1,AUT,Austria,2018,290.71,2796.09,2377.60,180.76,2990.13,4012.92,153.52,667.12,574.45,10070.66,46515.29,23789.52,23843.66
2,BEL,Belgium,2018,462.33,3644.00,3639.36,525.85,4167.82,4412.44,186.64,987.24,724.70,11245.36,68982.70,30133.97,29738.61
3,CHE,Switzerland,2018,478.74,2294.88,3265.60,357.89,2700.85,1259.73,107.57,984.79,480.45,7836.67,24812.68,19767.15,20506.09
4,CZE,Czech Republic,2018,343.06,2405.10,1952.99,339.62,1642.54,3153.13,250.21,747.35,558.27,5057.88,16932.08,17144.94,17520.80
5,DEU,Germany,2018,4674.19,13979.88,18298.10,2810.74,25054.11,31644.36,1702.33,6840.17,4524.62,86244.30,318714.45,195505.93,203140.61
6,DNK,Denmark,2018,374.64,1041.54,2073.20,130.04,1985.25,2676.76,67.18,300.35,545.41,7148.89,15338.55,16440.81,16619.48
7,ESP,Spain,2018,1657.86,7067.70,7404.52,1602.33,10324.55,11046.09,814.13,3418.98,2019.34,30716.06,210269.33,76717.45,72112.74
8,EST,Estonia,2018,95.34,203.14,271.95,35.06,182.90,236.06,16.64,88.99,98.11,610.68,587.84,1851.77,1825.85
9,FIN,Finland,2018,331.90,1134.14,1501.95,53.51,2088.51,1870.35,86.46,301.19,386.83,6572.99,18393.32,14177.50,13967.14


In [51]:
JSON_NAMES_MAP = {
    'Code': 'countryCode',
    'Country': 'country',
    'Time': 'year',
    'economic affairs': 'economicAffairs',
    'environment protection': 'environmentProtection',
    'general public services': 'generalPublicServices',
    'housing and community amenities': 'housingAndCommunityAmenities',
    'public order and safety': 'publicOrderAndSafety',
    'recreation, culture and religion': 'recreationCultureAndReligion',
    'social protection': 'socialProtection',
    'Gross debt of general government': 'totalDebt',
    'Total expenditure of general government': 'totalBudget',
    'Total general government (GG) revenue': 'totalRevenue'
}

def clean_col_names(col_name):
    trimmed_name = col_name.replace(
        'General government expenditure by function, ', ''
    ).replace(', percentage of GDP', '')
    
    return JSON_NAMES_MAP.get(trimmed_name) or trimmed_name

clean_df = budgets_dollar.rename(columns=clean_col_names)
clean_df

,countryCode,country,year,defence,economicAffairs,education,environmentProtection,generalPublicServices,health,housingAndCommunityAmenities,publicOrderAndSafety,recreationCultureAndReligion,socialProtection,totalDebt,totalBudget,totalRevenue
0,AUS,Australia,2018,2130.69,4899.34,7151.97,1189.57,6112.27,9476.24,799.88,2383.37,979.71,13786.13,89783.66,49678.94,48541.24
1,AUT,Austria,2018,290.71,2796.09,2377.60,180.76,2990.13,4012.92,153.52,667.12,574.45,10070.66,46515.29,23789.52,23843.66
2,BEL,Belgium,2018,462.33,3644.00,3639.36,525.85,4167.82,4412.44,186.64,987.24,724.70,11245.36,68982.70,30133.97,29738.61
3,CHE,Switzerland,2018,478.74,2294.88,3265.60,357.89,2700.85,1259.73,107.57,984.79,480.45,7836.67,24812.68,19767.15,20506.09
4,CZE,Czech Republic,2018,343.06,2405.10,1952.99,339.62,1642.54,3153.13,250.21,747.35,558.27,5057.88,16932.08,17144.94,17520.80
5,DEU,Germany,2018,4674.19,13979.88,18298.10,2810.74,25054.11,31644.36,1702.33,6840.17,4524.62,86244.30,318714.45,195505.93,203140.61
6,DNK,Denmark,2018,374.64,1041.54,2073.20,130.04,1985.25,2676.76,67.18,300.35,545.41,7148.89,15338.55,16440.81,16619.48
7,ESP,Spain,2018,1657.86,7067.70,7404.52,1602.33,10324.55,11046.09,814.13,3418.98,2019.34,30716.06,210269.33,76717.45,72112.74
8,EST,Estonia,2018,95.34,203.14,271.95,35.06,182.90,236.06,16.64,88.99,98.11,610.68,587.84,1851.77,1825.85
9,FIN,Finland,2018,331.90,1134.14,1501.95,53.51,2088.51,1870.35,86.46,301.19,386.83,6572.99,18393.32,14177.50,13967.14


In [53]:
json.loads(json.dumps(clean_df.to_dict('records')))

[{'country': 'Australia',
  'countryCode': 'AUS',
  'defence': 2130.69,
  'economicAffairs': 4899.34,
  'education': 7151.97,
  'environmentProtection': 1189.57,
  'generalPublicServices': 6112.27,
  'health': 9476.24,
  'housingAndCommunityAmenities': 799.88,
  'publicOrderAndSafety': 2383.37,
  'recreationCultureAndReligion': 979.71,
  'socialProtection': 13786.13,
  'totalBudget': 49678.94,
  'totalDebt': 89783.66,
  'totalRevenue': 48541.24,
  'year': 2018},
 {'country': 'Austria',
  'countryCode': 'AUT',
  'defence': 290.71,
  'economicAffairs': 2796.09,
  'education': 2377.6,
  'environmentProtection': 180.76,
  'generalPublicServices': 2990.13,
  'health': 4012.92,
  'housingAndCommunityAmenities': 153.52,
  'publicOrderAndSafety': 667.12,
  'recreationCultureAndReligion': 574.45,
  'socialProtection': 10070.66,
  'totalBudget': 23789.52,
  'totalDebt': 46515.29,
  'totalRevenue': 23843.66,
  'year': 2018},
 {'country': 'Belgium',
  'countryCode': 'BEL',
  'defence': 462.33,
  '

In [49]:
totals = clean_df.filter(regex='^total')
budgets = clean_df.select_dtypes(float).drop(totals.columns, axis=1)

totals.assign(budgets = budgets.apply(sum, axis=1), country = clean_df['country'])

,totalDebt,totalBudget,totalRevenue,budgets,country
0,22104.04,19676.48,19012.85,18978.60,Australia
1,22934.30,20505.11,20074.95,19780.74,Australia
2,23178.93,21067.50,21494.66,20530.88,Australia
3,22578.58,22406.06,22920.74,21776.96,Australia
4,21687.28,23820.31,24456.83,23050.67,Australia
5,21504.65,25074.98,26269.06,24032.90,Australia
6,22624.11,26899.54,28181.25,25668.13,Australia
7,23095.91,28605.25,29199.52,27093.04,Australia
8,25513.69,31510.89,28319.97,29814.98,Australia
9,34864.35,34634.94,29616.60,31919.66,Australia
